In [2]:
import os

topic_file = os.getenv("TOPIC_PATH")
users_file = os.getenv("USERS_PATH")
ratings_file = os.getenv("RATINGS_PATH")

#topic_file = "/data/movie_data/movies.csv"
#users_file = "/data/movie_data/users.csv"
#ratings_file = "/data/movie_data/rating.csv"

In [41]:
import pandas as pd
import numpy as np
topics = pd.read_csv(topic_file)

In [4]:
ratings = pd.read_csv(ratings_file)
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
users = pd.read_csv(users_file)
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [47]:
user_rating = pd.merge(users,ratings,on='UserID')
user_rating_mv = pd.merge(user_rating, topics, left_on="MovieID", right_on="ID") 

In [49]:
user_rating_mv.head()

,UserID,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,ID,NAME,CATEGORY
0,1,F,1,10,48067,1193,5,978300760,1193,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,1193,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,1193,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,1193,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,1193,One Flew Over the Cuckoo's Nest (1975),Drama


In [51]:
final_df_matrix = user_rating_mv.pivot(index='UserID',
                                 columns='MovieID',
                                 values='Rating').fillna(0)

In [52]:
user_ratings_mean = np.mean(final_df_matrix.values, axis=1)
ratings_demeaned = final_df_matrix.values - user_ratings_mean.reshape(-1, 1)

In [55]:
n_users = user_rating_mv['UserID'].nunique()
n_movies = user_rating_mv['MovieID'].nunique()
sparsity = round(1.0 - user_rating_mv.shape[0] / float(n_users * n_movies), 3)
print('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

The sparsity level of MovieLens1M dataset is 95.5%


In [56]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(ratings_demeaned, k=50)  # Number of singular values and vectors to compute